In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-14 17:42:10 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'


/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-07-14 17:42:10 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100-SXM4-80GB'
2025-07-14 17:42:10 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-07-14 17:42:12 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-14 17:42:12 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-14 17:42:12 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-14 17:42:12 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-14 17:42:12 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-14 17:42:12 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-07-14 17:42:12 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-14 17:42:13 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards: 100%|██████████| 30/30 [00:24<00:00,  1.24it/s]

2025-07-14 17:42:40 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0


2025-07-14 17:42:40 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [6]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "test_72"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>10<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [7]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-14 17:42:50 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])


2025-07-14 17:42:50 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-14 17:42:51 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-14 17:42:51 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-14 17:42:51 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-14 17:42:51 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-14 17:42:51 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-14 17:42:51 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-

In [68]:
from src.functional import generate_with_patch


# SELECT ONE TASK
prompt_template = """Which person from the following list has the profession in common with {}?
{}.
Ans:"""

# pivot_subj = "Celine Dion"
# entity_list = ["Ryan Reynolds", "Claude Monet", "Albert Einstein", "Taylor Swift", "Pablo Picasso", "Barack Obama", "J.K. Rowling", "Pierre Dubois"]

# pivot_subj = "Jim Henson"
# entity_list = [
#     "Celine Dion",
#     "Taylor Swift",
#     "Ryan Reynolds",
#     "Claude Monet",
#     "Albert Einstein",
#     "Pablo Picasso",
#     "Bil Baird",
#     "Barack Obama",
#     "J.K. Rowling",
#     "Pierre Dubois",
#     "Sachin Tendulkar",
#     "Hugh Jackman",
#     "Jackie Chan",
#     "Jet Li",
#     "Diego Maradona",
#     "Lionel Messi",
#     "Frida Kahlo",
#     "Diego Rivera",
# ]

# patch_subj = "Justin Trudeau"
# patch_subj = "Hugh Jackman"
# patch_subj = "Carl Sagan"
# patch_subj = "Ricky Ponting"
patch_subj = "George R. R. Martin"

patch_list = [
    "Robin Hobb",
    "Celine Dion",
    "Taylor Swift",
    "Tom Cruise",
    "Barack Obama",
    "Albert Einstein",
    "Brian Lara",
]

# prompt = prompt_template.format(pivot_subj, ",".join(entity_list))

# generate_with_patch(
#     mt=mt,
#     inputs=prompt,
#     max_new_tokens=30,
#     n_gen_per_prompt=1,
#     do_sample=False,
#     remove_prefix=True,
# )

In [69]:
from src.functional import get_hs, interpret_logits

locations = [(layer_name, -1) for layer_name in mt.layer_names]
logit_location = (mt.lm_head_name, -1)

patch_prompt = prompt_template.format(patch_subj, ",".join(patch_list))

patch_hs = get_hs(
    mt = mt,
    input=patch_prompt,
    locations=locations + [logit_location],
    return_dict=True
)

logit = patch_hs[logit_location]
interpret_logits(
    logits = logit,
    tokenizer=mt
)

[PredictedToken(token=' Robin', prob=0.73828125, logit=19.75, token_id=17582, metadata=None),
 PredictedToken(token='Robin', prob=0.11328125, logit=17.875, token_id=77771, metadata=None),
 PredictedToken(token=' George', prob=0.0537109375, logit=17.125, token_id=10058, metadata=None),
 PredictedToken(token=' The', prob=0.019775390625, logit=16.125, token_id=578, metadata=None),
 PredictedToken(token=' None', prob=0.0174560546875, logit=16.0, token_id=2290, metadata=None)]

In [72]:
# pivot_subj = "Andy Murray"
pivot_subj = "Hugh Jackman"

entity_list = [
    "Issac Newton",
    "Serena Williams",
    "Sachin Tendulkar",
    "Tolkien",
    "Michael Jackson",
    "Ryan Reynolds",
    "Marie Curie",
]

clean_hs = get_hs(
    mt=mt,
    input=prompt_template.format(pivot_subj, ",".join(entity_list)),
    locations=locations + [logit_location],
    return_dict=True,
)
logit = clean_hs[logit_location]
interpret_logits(logits=logit, tokenizer=mt)

[PredictedToken(token=' Ryan', prob=0.66015625, logit=19.25, token_id=13960, metadata=None),
 PredictedToken(token='Ryan', prob=0.146484375, logit=17.75, token_id=49546, metadata=None),
 PredictedToken(token=' Michael', prob=0.037109375, logit=16.375, token_id=8096, metadata=None),
 PredictedToken(token=' The', prob=0.037109375, logit=16.375, token_id=578, metadata=None),
 PredictedToken(token=' Hugh', prob=0.02392578125, logit=15.9375, token_id=30206, metadata=None)]

In [73]:
from src.functional import PatchSpec

clean_prompt = prompt_template.format(pivot_subj, ",".join(entity_list))

for layer_name in mt.layer_names[::2]:
    int_logit = get_hs(
        mt = mt,
        input = clean_prompt,
        patches = [PatchSpec(
            location=(layer_name, -1),
            patch=patch_hs[(layer_name, -1)],
            strategy="replace"
        )],
        locations = [logit_location],
        return_dict=False
    ).squeeze(0)

    pred = interpret_logits(
        logits=int_logit,
        tokenizer=mt,
    )
    print(f'{layer_name}: {[str(p) for p in pred]}')

model.layers.0: ['" Ryan"[13960] (p=0.645, logit=19.250)', '"Ryan"[49546] (p=0.163, logit=17.875)', '" The"[578] (p=0.036, logit=16.375)', '" Michael"[8096] (p=0.032, logit=16.250)', '" Hugh"[30206] (p=0.025, logit=16.000)']
model.layers.2: ['" Ryan"[13960] (p=0.660, logit=19.250)', '"Ryan"[49546] (p=0.147, logit=17.750)', '" The"[578] (p=0.037, logit=16.375)', '" Michael"[8096] (p=0.033, logit=16.250)', '" Hugh"[30206] (p=0.024, logit=15.938)']
model.layers.4: ['" Ryan"[13960] (p=0.660, logit=19.250)', '"Ryan"[49546] (p=0.147, logit=17.750)', '" The"[578] (p=0.037, logit=16.375)', '" Michael"[8096] (p=0.033, logit=16.250)', '" Hugh"[30206] (p=0.026, logit=16.000)']
model.layers.6: ['" Ryan"[13960] (p=0.660, logit=19.250)', '"Ryan"[49546] (p=0.147, logit=17.750)', '" The"[578] (p=0.037, logit=16.375)', '" Michael"[8096] (p=0.033, logit=16.250)', '" Hugh"[30206] (p=0.026, logit=16.000)']
model.layers.8: ['" Ryan"[13960] (p=0.656, logit=19.250)', '"Ryan"[49546] (p=0.146, logit=17.750)', 

In [ ]:
# Exclusion
prompt_template = """Which person from the following list does not have the profession in common with {}?
{}.
Ans:"""

# pivot_subj = "Hugh Jackman"
# entity_list = [
#     "Tom Cruise",
#     "Brad Pitt",
#     "Ryan Reynolds",
#     "Celine Dion",
#     "Tom Hanks",
#     "Scarlett Johansson",
# ]

# pivot_subj = "Michael Jordan"
# entity_list = [
#     "Sachin Tendulkar",
#     "Roger Federer",
#     "Lionel Messi",
#     "Kobe Bryant",
#     "Tom Brady",
#     "Hugh Jackman",
#     "Serena Williams",
#     "Mohamed Salah",
#     "Tiger Woods",
#     "Jesse Owens",
#     "Cristiano Ronaldo",
#     "Mike Tyson",
#     "Usain Bolt",
#     "Michael Phelps",
#     "LeBron James",
# ]

# prompt_template = """Which person from the following list didn't graduate from the same school as {}
# {}.
# Ans:"""

# pivot_subj = "Steve Martin"
# entity_list = [
#    "Francis Ford Coppola",
#    "Kareem Abdul-Jabbar",
#    "Jackie Robinson",
#    "Jim Morrison",
#    "Susan Wojcicki",
#    "Carol Burnett",
#    "George Lucas",
#    "Arthur Ashe",
#    "Mayim Bialik",
#    "Troy Aikman",
#    "Randy Schekman"
# ]

prompt_template = """Which object in this list is does not share the same color as {}?
{}.
Ans:"""

# pivot_subj = "banana"
# entity_list = ["pineapple", "sunflower", "canary", "blood", "mustard", "taxi", "corn"]
pivot_subj = "fire extinguisher"
patch_list = ["stop sign", "apple", "tomato", "snow", "traffic light", "fire truck"]


prompt = prompt_template.format(pivot_subj, ",".join(patch_list))
generate_with_patch(
    mt=mt,
    inputs=prompt,
    max_new_tokens=30,
    n_gen_per_prompt=1,
    do_sample=False,
    remove_prefix=True,
)

/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[' snow.\nThe color of fire extinguisher is red. The objects in the list that are red are stop sign, apple, tomato, and fire truck']

In [ ]:
# NOVELTY: Find the odd one out

prompt_template = """Which person from the following list is different from the others?
{}.
Ans:"""

patch_list = []